**TOPIC: Non-Euclidean Feature Embedding for Proactive Industrial Resilience using an Adaptive Decoupled Graph Autoencoder**


**DESCRIPTION: **Our project introduces a novel unsupervised learning framework designed for enhancing the resilience and operational longevity of industrial assets. We present an Adaptive Decoupled Graph Autoencoder that leverages Non-Euclidean Feature Embedding to model complex, interconnected sensor data. Our approach overcomes the limitations of traditional methods by learning intricate spatio-temporal dependencies within industrial sensor networks. By decoupling spatial learning from temporal dynamics and incorporating adaptive mechanisms, the model effectively identifies subtle anomalies that signal impending faults. This proactive diagnosis capability aims to significantly improve Industrial Resilience by enabling timely maintenance interventions, thereby reducing downtime, operational costs, and catastrophic failures in diverse industrial environments. This framework is particularly suited for applications in Smart India Hackathon themes focused on intelligent automation and sustainable infrastructure.

In [ ]:
# Install PyTorch Geometric and dependencies
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl size=664632 sha256=4fe9bba9203467f8e04022273d61d48bec3deec5479ca789251c5c137522dcdf
  Stored in directory: /root/.cache/pip/wheels/84/20/50/44800723f57cd798630e77b3ec83bc80bd26a1e3dc3a672ef5
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl size=1222874 sha256=f6e3458c8a398cc04e88ccac4cb48e300e3dbc0eb

In [ ]:
# === IMPORTS ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse

# === LOAD KAGGLE DATASET ===
data = pd.read_csv("sensor.csv")
print("Shape:", data.shape)
print(data.head())

# SELECT SENSOR COLUMNS (drop timestamp if present)
sensor_cols = [c for c in data.columns if "sensor" in c]

# === NORMALIZATION ===
scaler = MinMaxScaler()
data[sensor_cols] = scaler.fit_transform(data[sensor_cols])

X = torch.tensor(data[sensor_cols].values, dtype=torch.float32)

# === BUILD GRAPH (CORRELATION > 0.5) ===
corr = data[sensor_cols].corr().values
adj = (corr > 0.5).astype(float)
edge_index, edge_weight = dense_to_sparse(torch.tensor(adj))

# === MODEL: Graph Autoencoder ===
class GraphAutoencoder(nn.Module):
    def __init__(self, in_feat):
        super().__init__()
        self.encoder = GCNConv(in_feat, 32)
        self.decoder = nn.Linear(32, in_feat)

    def forward(self, x, edge_index):
        z = torch.relu(self.encoder(x, edge_index))
        return self.decoder(z)

model = GraphAutoencoder(len(sensor_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# === UNSUPERVISED TRAINING ===
for epoch in range(30):
    optimizer.zero_grad()
    recon = model(X, edge_index)
    loss = loss_fn(recon, X)
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

# === ANOMALY SCORES (RECON ERROR) ===
recon = model(X, edge_index).detach().numpy()
error = np.mean((recon - X.numpy())**2, axis=1)

threshold = np.percentile(error, 95)
preds = (error > threshold).astype(int)

# === BUILD GROUND TRUTH LABELS ===
# ASSUME dataset has a column 'label' (0=normal, 1=fault)
if "label" in data.columns:
    labels = data["label"].values
else:
    # simulate labels near end of each hour of data
    labels = np.zeros(len(data))
    labels[-int(len(data) * 0.05):] = 1  # last 5% anomalies

# === REPORT METRICS ===
acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds)

print("Accuracy:", acc)
print("F1 Score:", f1)


Shape: (220320, 55)
   Unnamed: 0            timestamp  sensor_00  sensor_01  sensor_02  \
0           0  2018-04-01 00:00:00   2.465394   47.09201    53.2118   
1           1  2018-04-01 00:01:00   2.465394   47.09201    53.2118   
2           2  2018-04-01 00:02:00   2.444734   47.35243    53.2118   
3           3  2018-04-01 00:03:00   2.460474   47.09201    53.1684   
4           4  2018-04-01 00:04:00   2.445718   47.13541    53.2118   

   sensor_03  sensor_04  sensor_05  sensor_06  sensor_07  ...  sensor_43  \
0  46.310760   634.3750   76.45975   13.41146   16.13136  ...   41.92708   
1  46.310760   634.3750   76.45975   13.41146   16.13136  ...   41.92708   
2  46.397570   638.8889   73.54598   13.32465   16.03733  ...   41.66666   
3  46.397568   628.1250   76.98898   13.31742   16.24711  ...   40.88541   
4  46.397568   636.4583   76.58897   13.35359   16.21094  ...   41.40625   

   sensor_44  sensor_45  sensor_46  sensor_47  sensor_48  sensor_49  \
0  39.641200   65.68287  

/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


Epoch 0, Loss: nan
Epoch 5, Loss: nan
Epoch 10, Loss: nan
Epoch 15, Loss: nan
Epoch 20, Loss: nan
Epoch 25, Loss: nan
Accuracy: 0.95
F1 Score: 0.0
